<h2>Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
import csv
import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
from arcgis.gis import GIS
import urllib
from geopy.geocoders import Nominatim
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as plt
import numpy as np
from pandas.io.json import json_normalize
import wget
from sklearn.preprocessing import MinMaxScaler

Using Beautiful Soup's library functions to extract Postal codes, Borough and Neighbourhood data from Wikipedia page 

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
html = BeautifulSoup(page.text, 'html.parser')
table = html.findAll('table',{"class":"sortable"})[0]
end_col = len(table.findAll('th'))
end_row = len(table.findAll('tr'))

Logic to extract data with html tags & appending them to empty list

In [3]:
row_data=[]
start_col = 0
start_row = 1   
tr = table.findAll(['tr'])[start_row:end_row]
th = table.find_all(['th'])[start_col:end_col]
th_data = [col.text.strip('\n') for col in th]
for cell in tr:
    td = cell.find_all('td')
    row = [i.text.replace('\n','') for i in td]
    if row[1] != "Not assigned":
        row_data.append(row)

Creating dataframe and setting the columns appropriately

In [4]:
df_toronto = pd.DataFrame(row_data) 
df_toronto.columns = th_data
df_toronto.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_toronto

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Using the URL provided in the assignment to get the co-ordinates based on the Postal Code.

In [5]:
df_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the coordinates dataframe with the Neighbourhoods dataframe

In [6]:
df_toronto = pd.merge(df_toronto, df_coordinates, on='PostalCode', how='left')
df_toronto

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Filtering the data from all the neighbourhoods to with Boroughs containing Toronto in it's name

In [7]:
toronto_data = df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### Create a map of Toronto with neighborhoods superimposed on top.

In [51]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'],toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Get & Set Foursquare credentials

In [52]:
CLIENT_ID = '5MG2XUM2YZXTCLG3MXCPVDB4BVGNNE5413AFDU54MRCY4SJD' # your Foursquare ID
CLIENT_SECRET = 'CZXPK4ILKGKBFBJTK1BXBWPC5IQCKACW11U4ZVVY34WBSZMY' # your Foursquare Secret
VERSION = '20200731' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5MG2XUM2YZXTCLG3MXCPVDB4BVGNNE5413AFDU54MRCY4SJD
CLIENT_SECRET:CZXPK4ILKGKBFBJTK1BXBWPC5IQCKACW11U4ZVVY34WBSZMY


In [53]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [54]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude']
neighborhood_longitude = toronto_data.loc[0, 'Longitude']
neighborhood_name = toronto_data.loc[0, 'Neighbourhood']
print(neighborhood_latitude, neighborhood_longitude, neighborhood_name)

43.6542599 -79.3606359 Regent Park, Harbourfront


#### Get the Four square API URL with relevant details like, longitude, latitude, client ID & Secret

In [55]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=5MG2XUM2YZXTCLG3MXCPVDB4BVGNNE5413AFDU54MRCY4SJD&client_secret=CZXPK4ILKGKBFBJTK1BXBWPC5IQCKACW11U4ZVVY34WBSZMY&v=20200731&ll=43.6542599,-79.3606359&radius=500&limit=100'

#### Get the results in JSON format

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f28dd6ccc2d4f2e205e25fa'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

In [57]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Get Venues and categories of venues with the co-ordinates

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
2157,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Royal Canadian Legion #210,43.628855,-79.518903,Social Club
2158,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2159,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2160,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


In [63]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### Groupby dataframe by neighborhood

In [64]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,3,3,3,3,3,3


In [65]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


#### One hot coding toe get the each venue type's occurence.

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_onehot.shape

(2162, 272)

In [68]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.037037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.029412,0.0,0.0,0.0,0.000000
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.000000


In [69]:
toronto_grouped.shape

(96, 272)

#### Get top 10 venues

In [70]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0                   Clothing Store   0.2
1                           Lounge   0.2
2                     Skating Rink   0.2
3                   Breakfast Spot   0.2
4        Latin American Restaurant   0.2
5               Mexican Restaurant   0.0
6  Molecular Gastronomy Restaurant   0.0
7       Modern European Restaurant   0.0
8                Mobile Phone Shop   0.0
9               Miscellaneous Shop   0.0


----Alderwood, Long Branch----
                      venue  freq
0               Pizza Place  0.25
1                  Pharmacy  0.12
2                       Pub  0.12
3            Sandwich Place  0.12
4               Coffee Shop  0.12
5              Skating Rink  0.12
6                       Gym  0.12
7            Medical Center  0.00
8  Mediterranean Restaurant  0.00
9               Men's Store  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0

9               Metro Station  0.00


----Garden District, Ryerson----
                 venue  freq
0       Clothing Store  0.10
1          Coffee Shop  0.08
2                 Café  0.04
3  Japanese Restaurant  0.03
4      Bubble Tea Shop  0.03
5   Italian Restaurant  0.03
6       Cosmetics Shop  0.03
7          Pizza Place  0.02
8     Ramen Restaurant  0.02
9                Hotel  0.02


----Glencairn----
                             venue  freq
0                             Park  0.25
1                      Pizza Place  0.25
2              Japanese Restaurant  0.25
3                              Pub  0.25
4        Middle Eastern Restaurant  0.00
5  Molecular Gastronomy Restaurant  0.00
6       Modern European Restaurant  0.00
7                Mobile Phone Shop  0.00
8               Miscellaneous Shop  0.00
9               Mexican Restaurant  0.00


----Golden Mile, Clairlea, Oakridge----
                      venue  freq
0                    Bakery   0.2
1                  Bus Line  

9   Mexican Restaurant  0.06


----Northwest, West Humber - Clairville----
                        venue  freq
0                   Drugstore   0.5
1         Rental Car Location   0.5
2                 Yoga Studio   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0
5          Miscellaneous Shop   0.0
6   Middle Eastern Restaurant   0.0
7          Mexican Restaurant   0.0
8               Metro Station   0.0
9         Monument / Landmark   0.0


----Northwood Park, York University----
                        venue  freq
0              Massage Studio  0.14
1        Caribbean Restaurant  0.14
2               Metro Station  0.14
3                         Bar  0.14
4                 Coffee Shop  0.14
5      Furniture / Home Store  0.14
6          Miscellaneous Shop  0.14
7                 Yoga Studio  0.00
8  Modern European Restaurant  0.00
9         Moroccan Restaurant  0.00


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East

                             venue  freq
0            Portuguese Restaurant  0.25
1                      Pizza Place  0.25
2                      Coffee Shop  0.25
3                     Hockey Arena  0.25
4                      Yoga Studio  0.00
5  Molecular Gastronomy Restaurant  0.00
6       Modern European Restaurant  0.00
7                Mobile Phone Shop  0.00
8               Miscellaneous Shop  0.00
9        Middle Eastern Restaurant  0.00


----West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale----
                             venue  freq
0                    Jewelry Store   1.0
1                      Yoga Studio   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0
5               Miscellaneous Shop   0.0
6        Middle Eastern Restaurant   0.0
7               Mexican Restaurant   0.0
8                    Metro Station   0.0
9                           Lounge   0.0


----Westmount--

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Get top 100 venues

In [72]:
num_top_venues = 100

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Clothing Store,Donut Shop,Diner,Discount Store,Distribution Center,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Food Court
1,"Alderwood, Long Branch",Pizza Place,Skating Rink,Sandwich Place,Gym,Pub,Coffee Shop,Pharmacy,Department Store,Dessert Shop,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Sushi Restaurant,Frozen Yogurt Shop,Ice Cream Shop,Middle Eastern Restaurant,Supermarket,Deli / Bodega,Mobile Phone Shop,...,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Food Court,Airport
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant,Pharmacy,Butcher,Pub,...,Bookstore,Boat or Ferry,Board Shop,Bistro,Bike Shop,Belgian Restaurant,Beer Store,Beer Bar,Bed & Breakfast,Beach


In [73]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 3, 1,
       0, 1, 0, 1, 1, 1, 1, 0])

In [74]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

neighborhoods_venues_sorted

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,...,Climbing Gym,Church,Asian Restaurant,Basketball Court,Baseball Stadium,Baseball Field,Bar,Video Store,Vietnamese Restaurant,Bagel Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Yoga Studio,Portuguese Restaurant,...,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Art Museum,Aquarium,Antique Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,...,Donut Shop,Creperie,Filipino Restaurant,Dim Sum Restaurant,Fish & Chips Shop,Fish Market,Coworking Space,Cuban Restaurant,Candy Store,College Stadium
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Restaurant,Coffee Shop,Café,Clothing Store,...,Auto Workshop,Bike Shop,Beer Store,Bed & Breakfast,Beach,Basketball Stadium,Basketball Court,Baseball Stadium,Baseball Field,Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Pub,Health Food Store,Doner Restaurant,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Food Court,Airport
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Restaurant,Bakery,...,Belgian Restaurant,Beer Store,Bed & Breakfast,Basketball Court,Baseball Stadium,Baseball Field,Bar,Bank,Baby Store,Auto Workshop
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,...,Drugstore,Doner Restaurant,Dog Run,Distribution Center,Theater,Climbing Gym,Chocolate Shop,Fried Chicken Joint,Beach,Basketball Court
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Baby Store,...,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Gym,Clothing Store,...,Fabric Shop,Sports Bar,Ethiopian Restaurant,Stadium,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Doner Restaurant,Caribbean Restaurant,Dog Run
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Park,Music Venue,...,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service


#### Create a new map with all the neighbourhoods

In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Clusters displayed with venues in each of the neighbourhoods

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
33,Downtown Toronto,0,Park,Playground,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Food Court


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Café,Theater,Restaurant,Breakfast Spot,...,Climbing Gym,Church,Asian Restaurant,Basketball Court,Baseball Stadium,Baseball Field,Bar,Video Store,Vietnamese Restaurant,Bagel Shop
1,Downtown Toronto,1,Coffee Shop,Diner,Yoga Studio,Portuguese Restaurant,Beer Bar,Bar,Bank,Mexican Restaurant,...,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Art Museum,Aquarium,Antique Shop
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Electronics Store,...,Donut Shop,Creperie,Filipino Restaurant,Dim Sum Restaurant,Fish & Chips Shop,Fish Market,Coworking Space,Cuban Restaurant,Candy Store,College Stadium
3,Downtown Toronto,1,Restaurant,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Cocktail Bar,American Restaurant,Art Gallery,...,Auto Workshop,Bike Shop,Beer Store,Bed & Breakfast,Beach,Basketball Stadium,Basketball Court,Baseball Stadium,Baseball Field,Bar
4,East Toronto,1,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Food Court,Airport
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,...,Belgian Restaurant,Beer Store,Bed & Breakfast,Basketball Court,Baseball Stadium,Baseball Field,Bar,Bank,Baby Store,Auto Workshop
6,Downtown Toronto,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Department Store,...,Drugstore,Doner Restaurant,Dog Run,Distribution Center,Theater,Climbing Gym,Chocolate Shop,Fried Chicken Joint,Beach,Basketball Court
7,Downtown Toronto,1,Grocery Store,Café,Park,Baby Store,Coffee Shop,Diner,Restaurant,Italian Restaurant,...,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Gym,Clothing Store,Hotel,Restaurant,Bar,Steakhouse,...,Fabric Shop,Sports Bar,Ethiopian Restaurant,Stadium,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Doner Restaurant,Caribbean Restaurant,Dog Run
9,West Toronto,1,Bakery,Pharmacy,Park,Music Venue,Brewery,Café,Bar,Bank,...,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service


In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue


In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
